## GUI

In [26]:
from ipywidgets import widgets, Layout, interact, interactive, fixed, interact_manual, HBox, VBox
from IPython.display import Markdown
import datetime

%run Fundus_clean.ipynb

image_array_basic = ['01_h.jpg', '02_h.jpg', '03_h.jpg', '04_h.jpg', '05_h.jpg', 
                   '06_h.jpg', '07_h.jpg', '08_h.jpg', '09_h.jpg', '10_h.jpg', 
                   '11_h.jpg', '12_h.jpg', '13_h.jpg', '14_h.jpg', '15_h.jpg']

learning_array = ['Image_01L.jpg', 'Image_01R.jpg', 'Image_02L.jpg', 'Image_02R.jpg', 'Image_03L.jpg', 'Image_03R.jpg', 
                 'Image_04L.jpg', 'Image_04R.jpg', 'Image_05L.jpg', 'Image_05R.jpg', 'Image_06L.jpg', 'Image_06R.jpg', 
                 'Image_07L.jpg', 'Image_07R.jpg', 'Image_08L.jpg', 'Image_08R.jpg', 'Image_09L.jpg', 'Image_09R.jpg', 
                 'Image_10L.jpg', 'Image_10R.jpg', 'Image_11L.jpg', 'Image_11R.jpg', 'Image_12L.jpg', 'Image_12R.jpg', 
                 'Image_13L.jpg', 'Image_13R.jpg', 'Image_14L.jpg', 'Image_14R.jpg']
        
not_used_in_ML_small = ['Image_12R.jpg', 'Image_13L.jpg', 'Image_13R.jpg', 'Image_14L.jpg', 'Image_14R.jpg']
not_used_in_ML_big = ['11_h.jpg', '12_h.jpg', '13_h.jpg', '14_h.jpg', '15_h.jpg',
                      'Image_12R.jpg', 'Image_13L.jpg', 'Image_13R.jpg', 'Image_14L.jpg', 'Image_14R.jpg']

c = ImageReader()
f = BasicProcessing(c)
d = ImageLearning()
u = DeepLearning()
model = []
arr = []

def on_save_clicked_basic_input(x):
    print(f"Your choice: {filename_input_basic_input.value}")
    c.upload_image(filename_input_basic_input.value, "01_h_mask.tif")
    c.show_everycolor_channel()
    f = BasicProcessing(c)
    
def on_click_start_basicProcessing(x):
    print("Starting processing...")
    f.find_fundus()
    f.end_of_processing(filename_input_basic_input.value)
    
def on_click_start_basicProcessing_button(x):
    print("Accuracy: %f \nSensitivity: %f \nSpecificity: %f \nGeo mean metric: %f\n"%f.statistic())
    

def on_click_save_learning_set(x):
    print(f"Your choice: {select_learning_set.value}")
    global model 
    choose_image_knn = widgets.Dropdown(
        layout = Layout(width='350px'),
        description = 'filename',
        value = not_selected()[0],
        options = not_selected()
    )
    part = d.part_img(select_learning_set.value)
    model = d.model(part[1], part[0])
    
def on_click_start_knn(x):
    print("Accuracy: %f \nSensitivity: %f \nSpecificity: %f \nGeo mean metric: %f\n"%d.statistic())

def on_click_save_image_knn(x):
    d.image_model(model, choose_image_knn.value)

def on_click_start_ML(x):
    print("Accuracy: %f \nSensitivity: %f \nSpecificity: %f \nGeo mean metric: %f\n"%u.statistic())
    

def on_click_choose_model_button_file(x):
    global choose_model
    global choose_model_button
    choose_model = widgets.Dropdown(
        layout = Layout(width='350px'),
        description = 'filename',
        value = every_file(choose_model_file.value)[0],
        options = every_file(choose_model_file.value)
    ) 

    choose_model_button = widgets.Button(
        description = "Save filename",
        style = {'button_color':'lightpink'}
    )

    choose_model_button.on_click(on_click_choose_model_button)

    interactive_full_box_choose_model = HBox(
        [
            choose_model, 
            choose_model_button
        ]
    )
    display(interactive_full_box_choose_model)

def on_click_choose_model_button(x):
    global arr
    global choose_img
    global choose_img_button
    arr = not_used_in_ML_small
    if choose_model_file.value[:5] != "small":
        arr = not_used_in_ML_big
    
    choose_img = widgets.Dropdown(
        layout = Layout(width='350px'),
        description = 'filename',
        value = arr[0],
        options = arr
    ) 

    choose_img_button = widgets.Button(
        description = "Save image filename",
        style = {'button_color':'lightpink'}
    )

    choose_img_button.on_click(on_click_choose_img_button)

    interactive_full_box_choose_model = HBox(
        [
            choose_img, 
            choose_img_button
        ]
    )
    display(interactive_full_box_choose_model)
    
def on_click_choose_img_button(x):
    print("models/" + choose_model_file.value + "/", 
                choose_model.value, 
                choose_img.value)
    u.use_model("models/" + choose_model_file.value + "/", 
                choose_model.value, 
                choose_img.value)
    
    
def not_selected():
    global learning_array
    not_sel = []
    for img in learning_array:
        if img not in select_learning_set.value:
            not_sel.append(img)
    return not_sel

def every_dir():
    dirs = []
    rootdir = 'models/'
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            dirs.append(file)
    return dirs

def every_file(child_dir, rootdir = 'models/'):
    files = []
    for file in os.listdir(rootdir + child_dir + "/"):
        if file[-3:] != "txt":
            files.append(file)
    return files



filename_input_basic_input = widgets.Dropdown(
    layout = Layout(width='350px'),
    description = 'filename',
    value = '01_h.jpg',
    options = image_array_basic
)
save_button_basic_input = widgets.Button(
    description = "Save parameters",
    style = {'button_color':'lightpink'}
)
save_button_basic_input.on_click(on_save_clicked_basic_input)

interactive_full_box_basic_input = HBox(
    [
        filename_input_basic_input, 
        save_button_basic_input
    ]
)


start_basicProcessing_button = widgets.Button(
    description = "Start processing",
    style = {'button_color':'lightpink'}
)

start_basicProcessing_button.on_click(on_click_start_basicProcessing)

show_stats_basicProcessing_button = widgets.Button(
    description = "Show statistics",
    style = {'button_color':'lightpink'}
)

show_stats_basicProcessing_button.on_click(on_click_start_basicProcessing_button)


select_learning_set = widgets.SelectMultiple(
    layout = Layout(width = '120px', height = '460px'),
    options = learning_array,
    value = ['Image_01L.jpg'],
    description='',
    disabled=False
)

save_learning_set = widgets.Button(
    description = "Save learning set",
    style = {'button_color':'lightpink'}
)
save_learning_set.on_click(on_click_save_learning_set)

interactive_full_box_learning_set = HBox(
    [
        select_learning_set, 
        save_learning_set
    ]
)


choose_image_knn = widgets.Dropdown(
    layout = Layout(width='350px'),
    description = 'filename',
    value = not_selected()[0],
    options = not_selected()
)
save_image_knn = widgets.Button(
    description = "Save parameters",
    style = {'button_color':'lightpink'}
)
save_image_knn.on_click(on_click_save_image_knn)

interactive_full_box_knn_processing = HBox(
    [
        choose_image_knn, 
        save_image_knn
    ]
)


show_stats_knn = widgets.Button(
    description = "Show statistics",
    style = {'button_color':'lightpink'}
)

show_stats_knn.on_click(on_click_start_knn)

choose_model_file = widgets.Dropdown(
    layout = Layout(width='350px'),
    description = 'Directory',
    value = every_dir()[0],
    options = every_dir()
)

choose_model_button_file = widgets.Button(
    description = "Save dirname",
    style = {'button_color':'lightpink'}
)

choose_model_button_file.on_click(on_click_choose_model_button_file)

interactive_full_box_choose_model_file = HBox(
    [
        choose_model_file, 
        choose_model_button_file
    ]
)


choose_model = widgets.Dropdown() 

choose_model_button = widgets.Button()

choose_img = widgets.Dropdown() 

choose_img_button = widgets.Button()


show_stats_ML = widgets.Button(
    description = "Show statistics",
    style = {'button_color':'lightpink'}
)

show_stats_ML.on_click(on_click_start_ML)